<a href="https://colab.research.google.com/github/keremjn4/repo/blob/main/text_based_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from os import name
import datetime
import time
import threading
import random

##keys needs to be reconsidered.
class Player:
    def __init__(self, name, equipped_items):
        """Initialize the player's name, health, and inventory."""
        self.name = name
        self.health = 100
        self.inventory = []
        self.equipped_items = equipped_items
        self.alive = True
        self.basedamage = 3
        self.damage = self.basedamage + next((item.damage for item in self.equipped_items if item.damage > 0), 0,)
        self.critical_chance_multiplier = 1
        self.critical_chance = 0.01*self.critical_chance_multiplier
        self.critical_damage_multiplier = random.randint(2,3)
        self.incombat = False
    def search(self, environment,item):
        """Search the current room for items."""
        environment_items_names = ", ".join([item.name for item in environment.items])
        if environment.items:
           print(f"You see the following items: {environment_items_names}")
        else:
           print("You don't see anything worthy")

    def add_to_inventory(self, environment, item):
        if not environment.items:
            print("There are no items to pick up.")
            return
        """Add an item to the player's inventory."""
        item_to_take = input("Which item you wan't to take?").lower()
        item_to_take_list = [item.strip() for item in item_to_take.split(",")]

        for i in item_to_take_list:
            item_actual = next((item for item in environment.items if item.name == i), None)
            if len(self.inventory) >= 4 and item_actual.id !=1:
                print("Your hands are full. You can't hold more items.")
                break

            elif item_actual in environment.items and item_actual.id != 1:
                self.inventory.append(item_actual)
                environment.items.remove(item_actual)
                print(f"You picked up {item_actual.name}.")
            elif item_actual in environment.items and item_actual.id == 1:
                 self.inventory.append(item_actual)
                 environment.items.remove(item_actual)
                 print(f"You picked up {item_actual.name}.")
            else:
                print(f"You can't take {i}.")

    def drop_item(self, environment):
        """Drop an item from the player's inventory."""
        item_to_drop = input("which items you want to drop?").lower()
        item_to_drop_list = item_to_drop.split(",")
        for item in item_to_drop_list:
            if item in self.inventory:
                self.inventory.remove(item)
                environment.items.append(item)
                print(f"You dropped {item}.")
            elif item in self.keys:
                self.keys.remove(item)
                environment.items.append(item)
                print(f"you dropped {item}.")
            else:
                print(f"You don't have {item} in your inventory.")

    def show_inventory(self, item):
        if not self.inventory:
            print("Your inventory is empty.")
        else:
            if self.equipped_items:

                print(f"\nYour equipped items: {self.equipped_items}")
            elif not self.equipped_items:
                print("\nyou don't have any equipped items")
            inventory_items_names = ", ".join([item.name for item in self.inventory])
            print(f"\nYour inventory: {inventory_items_names}")

    def equip(self, environment, item):
        action = input("\nwhich item you wan't to equip?").lower()
        action_obj = next((item for item in self.inventory if item.name == action), 0)
        if action_obj == 0:
          print(f"{action} is not in your inventory")
        else:
          self.equipped_items.append(action_obj)
          action_obj.equipped = True
          print(f"{action_obj.name} is equipped")




    def combat(self, enemy, environment):
      while self.alive and enemy.alive:
        print(f"\n {self.name}'s Health: {self.health}")
        print(f"\n {enemy.name}'s Health: {enemy.health}")
        action = input("Choose an action: ").lower()
        if action == "attack":
            player_critical = random.random() <= self.critical_chance
            enemy_evade = random.random() <= enemy.evade_chance
            if enemy_evade:
               print(f"{enemy.name} evaded the attack!")
               enemy.enemy_attack(self)
               continue
            if player_critical:
               enemy.health -= self.critical_damage_multiplier*self.damage
               print(f"\n {self.name} executed CRITICAL HIT! {self.critical_damage_multiplier*self.damage} damage.")
            elif not player_critical:
              enemy.health -= self.damage
              print(f"\n {self.name} attacks {enemy.name} for {self.damage} damage!")
            if enemy.health <= 0:
              enemy.alive = False
              environment.enemies.remove(enemy)
              environment.enemy_list.remove(enemy.name)
              break
            if enemy.alive:
              enemy.enemy_attack(self)
        elif action == "flee":
             print(f"{self.name} flees from {enemy.name}")
             break
      if not self.alive:
         print(f"\n{self.name} has been defeated!")
      elif not enemy.alive:
         print(f"\n{enemy.name} has been defeated!")


    def interactions(self, environment):
        while True:
            # Prompt the player to choose an NPC to interact with
            choose_npc = input(f"Who do you want to interact with? Available: {environment.npc_list}\n {environment.enemy_list}").lower()

            if choose_npc in environment.npc_list:
                # If NPC exists, ask what the player wants to do
                player_interaction_decision = input(f"What do you want to do with {choose_npc}? ").lower()
                print(f"You chose to {player_interaction_decision} with {choose_npc}.")
                # Implement your interaction logic here
            elif choose_npc in environment.enemy_list:
                 enemy = next((enemy for enemy in environment.enemies if enemy.name.lower() == choose_npc), None)
                 if enemy and enemy.alive:
                    self.combat(enemy, environment)
                    break
                 else:
                    print(f"{choose_npc} is not here.")
            else:
                # If NPC does not exist, ask if they want to try again
                choose_another_npc = input(f"\n{choose_npc} is not here. Do you want to try someone else? (yes/no): ").lower()
                if choose_another_npc == "no":
                    print("\nYou decided not to interact with anyone.")
                    break
                elif choose_another_npc == "yes":
                    continue  #!!!!!!!!!Restart the loop to choose another NPC!!!!!!!!!!
                else:
                    print("Invalid input.")
                    break

    def list_of_actions(self):
       actions = ["search", "take", "inventory", "drop", "interact"]
       print(f"available actions: {actions}")
       return actions


class Enemy:
    def __init__(self, name, health, damage, _type):
        self.name = name
        self.health = health
        self.damage = damage
        self.alive = True
        self._type = _type
        self.evade_chance = 0.1
    def enemy_attack(self, player):
       if self.alive:
        player.health -= self.damage
        print(f"\n {self.name} attacks {player.name} for {self.damage} damage!")
        if player.health <= 0:
            player.health = 0
            player.alive = False
    def enemy_take_damage(self, player):
       if self.alive:
        self.health -= player.damage
        print(f"\n {self.name} takes {player.damage} damage!")
        if self.health <= 0:
            self.health = 0
            self.alive = False



class Environment:
    description_of_roomA = ("into a yeghegetsi. or what is left from this sacred site... \n crossing the narthex, felt the strong scent of burned flesh. \n you see a vartabed is looking at the ashes and the decayed corpses while he strokes his long, black beard..\n hood of his black robe covers his eyes. you can't read his emotions.")
    def __init__(self, description, items, npc=None, enemies=None):
        """Initialize a room with a description, items, and an optional NPC."""
        self.description = description
        self.npc = npc
        self.items = items
        print(f"\n You have entered {self.description}.")
        self.npc_list = self.npc.split(",")
        npc_names = ", ".join([self.npc for npc in self.npc_list])
        print(f"\n people in the room: {npc_names}")
        self.enemies = enemies or []
        self.enemy_list = [enemy.name for enemy in self.enemies]
        if self.enemies:
            enemy_names = ", ".join([enemy.name for enemy in self.enemies])
            print(f"\nEnemies in the room: {enemy_names}")

class item:
    def __init__(self, name, damage, description, id):
     self.name = name
     self.damage = damage
     self.description = description
     self.equipped = False
     self.id = id







class Game_Calendar:
    def __init__(self):
        self.months =  [{"name": "har", "days": 31, "monthno":1},
                        {"name": "safer", "days": 29,"monthno":2},
                        {"name": "rebiulevvel", "days": 31,"monthno":3},
                        {"name": "rebulahr", "days": 30,"monthno":4},
                        {"name": "cemaz", "days": 31,"monthno":5},
                        {"name": "cemazlahr", "days": 30,"monthno":6},
                        {"name": "rajap", "days": 31,"monthno":7},
                        {"name": "sabn", "days": 31,"monthno":8},
                        {"name": "nazamar", "days": 30,"monthno":9},
                        {"name": "sevbas", "days": 31,"monthno":10},
                        {"name": "zilkade", "days": 30,"monthno":11},
                        {"name": "zilhice", "days": 31,"monthno":12}]

        self.weekdays =  [{"name": "seme", "day":1},
                         {"name": "yewseme", "day":2},
                         {"name": "diseme", "day":3},
                         {"name": "seseme", "day":4},
                         {"name": "chʿorek", "day":5},
                         {"name": "hinkshaptʿi", "day":6},
                         {"name": "owrpat", "day":7}]

        self.current_year = 154
        self.current_month = self.months[0]
        self.current_day = self.weekdays[0]
        self.current_month_day = 1
        current_month_name = self.current_month["name"]
        current_day_name = self.current_day["name"]
        self.running = True
        self.start_time = time.time()
        self.sunpos = time.time()
        print(f"today is {self.current_month_day} / {current_month_name} / {self.current_year}, {current_day_name}")

    def time_advancer(self):
        time_elapsed = time.time() - self.start_time
        sun_position = time.time() - self.sunpos
        if sun_position >= 5:
            print("the sun has set and the moon has risen")
            self.sunpos = time.time()
            """Advance the time by one day."""
        if time_elapsed >= 10:  # 600 seconds simulate one day
            self.current_day = self.weekdays[self.current_day["day"] -1 + 1]
            self.current_month_day += 1
            self.start_time = time.time()
            print(f"sun rises. a new day has started:")
            if self.current_day["day"] == 7:
                self.current_day = self.weekdays[0]
            if self.current_month_day > self.current_month["days"]:
                self.current_month = self.months[self.current_month["monthno"] -1 + 1]
                self.current_month_day = 1
            if self.current_month["monthno"] == 12:
                self.current_year += 1
                self.current_month = self.months[0]



    def player_calendar(self):
       print(f"{self.current_month_day} / {self.current_month['name']} / {self.current_year}, {self.current_day['name']}")





def main():
    key =  item("key", 0, "key", 1)
    book = item("book", 0, "a book about carpentery",2)
    sword = item("sword", 20, "common sword", 3)
    player_name = input("Enter your player's name: ")
    calendar = Game_Calendar()
    player = Player(player_name,equipped_items=[])
    monster = [Enemy("monster", 100, 20, "common"),Enemy("monster", 100, 20,"common")]
    roomA = Environment(description=Environment.description_of_roomA, items=[sword, book, key], npc="vartabed", enemies= monster)

    while True:

        if not player.alive:
           print("game over")
           break

        calendar.time_advancer()
        print("\nWhat do you want to do?")
        action = input("Choose an action: ").lower()


        if action == "search":
            player.search(roomA,item)

        elif action == "take":
              player.add_to_inventory(roomA,item)

        elif action == "inventory":
              player.show_inventory(item)

        elif action == "drop":
              player.drop_item(roomA)

        elif action == "interact":
              player.interactions(roomA)

        elif action == "calendar":
              calendar.player_calendar()

        elif action == "help":
            player.list_of_actions()

        elif action == "equip":
            player.equip(roomA,item)

        elif action == "e":
            print("Exiting the game. Goodbye!")
            break

        else:
            print("you can't do that")




if __name__ == "__main__":
    main()


Enter your player's name: kerem
today is 1 / har / 154, seme

 You have entered into a yeghegetsi. or what is left from this sacred site... 
 crossing the narthex, felt the strong scent of burned flesh. 
 you see a vartabed is looking at the ashes and the decayed corpses while he strokes his long, black beard..
 hood of his black robe covers his eyes. you can't read his emotions..

 people in the room: vartabed

Enemies in the room: monster, monster

What do you want to do?
Choose an action: take
Which item you wan't to take?sword
You picked up sword.
the sun has set and the moon has risen

What do you want to do?
Choose an action: equip

which item you wan't to equip?sword
sword is equipped
the sun has set and the moon has risen
sun rises. a new day has started:

What do you want to do?
Choose an action: inventory


AttributeError: 'list' object has no attribute 'name'

In [ ]:
students = [
    {"name": "Alice", "grade": 85},
    {"name": "Bob", "grade": 87},
    {"name": "Bob", "grade": 90},
    {"name": "Charlie", "grade": 70},
]

# Find the first student with grade > 80
result = next((student for student in students if student["grade"] > 85), None)

print(result)

alice = students[0]
grade_alice = alice["grade"]
print(grade_alice)

{'name': 'Bob', 'grade': 87}
85


In [ ]:
weekdays =               [{"name": "seme", "day":1},
                         {"name": "yewseme", "day":2},
                         {"name": "diseme", "day":3},
                         {"name": "seseme", "day":4},
                         {"name": "chʿorek", "day":5},
                         {"name": "hinkshaptʿi", "day":6},
                         {"name": "owrpat", "day":7}]
today = weekdays[0]
nextday = today["day"]-1+2
print(today)
print(weekdays[nextday])


{'name': 'seme', 'day': 1}
{'name': 'diseme', 'day': 3}


In [ ]:
 def add_to_inventory(self, environment):
        if not environment.items:
            print("There are no items to pick up.")
            return
        """Add an item to the player's inventory."""
        item_to_take = input("Which item you wan't to take?").lower()
        item_to_take_list = [item.strip() for item in item_to_take.split(",")]

        for item in item_to_take_list:

            if len(self.inventory) >= 4 and "key" not in item:
                print("Your hands are full. You can't hold more items.")
                break

            if len(self.keys) >= 4 and "key" in item:
               print("your pockets are full. you can't carry more keys")

            elif item in environment.items and "key" not in item:
                self.inventory.append(item)
                environment.items.remove(item)
                print(f"You picked up {item}.")
            elif item in environment.items and "key" in item:
                 self.keys.append(item)
                 environment.items.remove(item)
                 print(f"You picked up {item}.")
            else:
                print(f"You can't take {item}.")


{'name': 'seme', 'day': 1}
{'name': 'yewseme', 'day': 2}
